In [1]:
import os
import shutil

original_db = "database/hotel_reservation_sample.db"
working_db = "database/hotel_reservation.db"
shutil.copy(original_db, working_db)

os.environ["DB"] = working_db

# Userstory 1
Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:

# Userstory 1.1
Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [29]:
import model
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    hotels = hotel_manager.search_hotels_by_city(city)
    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"{hotel.name} mit {hotel.stars} Sternen an der {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}")
        valid = True
    else:
        print("Keine Hotels gefunden")


Gefundene Hotels:
Bellevue Palace mit 5 Sternen an der Marktgasse 59, Bern, 3011


# Userstory 1.2
Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.


In [11]:
import model
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager


hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    stars = int(input("Geben Sie die mindest Anzahl Sterne an" ))
    hotels = hotel_manager.search_hotels_by_city_and_stars(city, stars)
    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"{hotel.name} mit {hotel.stars} Sternen an der {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}")
        valid = True
    else:
        print("Keine Hotels gefunden")


Gefundene Hotels:
Bellevue Palace mit 5 Sternen an der Marktgasse 59, Bern, 3011


# Userstory 3.1
Ich möchte neue Hotels zum System hinzufügen

## Input Daten vom Admin

* Daten wie Strasse, Hotelname, Anzahl Sterne und Ort(Addresse)




In [4]:
# Userstory 3.1
from datetime import date
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager
from model.hotel import Hotel
from model.address import Address

address_manager = AddressManager()
createded_address = address_manager.create_address("Teststrasse", "Zürich", 8001)
print(f"Neue Addresse: {createded_address.street}, {createded_address.city}, {createded_address.zip_code}")


hotel_manager = HotelManager()
h = Hotel(6, "Olympia", 3)
hotel = hotel_manager.create_hotel("Olympia", 3, createded_address)
print(f"Neues Hotel: {h.name}, Sterne: {h.stars}")

Neue Addresse: Teststrasse, Zürich, 8001
Neues Hotel: Olympia, Sterne: 3


# Userstory 4
Als Gast möchte ich ein Zimmer in einem bestimmten Hotel
buchen, um meinen Urlaub zu planen.


In [7]:
from business_logic.booking_manager import BookingManager
from model.guest import Guest
from model.room import Room
from model.room_type import RoomType
from datetime import date


guest = Guest(1, "John", "Doe", "mail")
room = Room(2, 202, 80.90)
room_type = RoomType(2, "Standard", 1)
room.room_type = room_type

booking_manager = BookingManager()
b = booking_manager.create_booking(guest, room, 1, date(2025, 1, 1), date(2025, 1, 3))
if room is None:
    raise ValueError("Ungültiges Zimmer: None")
else:
    print(f" Checkin Date: {b.check_in_date}, Amount: {b.total_amount}")

 Checkin Date: 2025-01-01, Amount: 161.8


User-Story 5:
Als Gast möchte ich nach meinem Aufenthalt eine Rechnung erhalten, damit ich einen Zahlungsnachweis habe. Hint: Fügt einen Eintrag in der "Invoice" Tabelle hinzu.



In [5]:
# Import model
from business_logic.invoice_manager import InvoiceManager
from model.invoice import Invoice
from data_access.invoice_dal import InvoiceDAL
from datetime import date
# Input Data from User
# Booking.bookingid: save in variable Bookingid
valid = False
while not valid:
    input_user = input("Geben Sie bitte Ihren Booking ID ein: ")
    if len(input_user) > 0:
        valid = True
    else:
        print("Bitte geben Sie einen Booking ID ein.")
print(input_user)

booking_id = int(input_user)
issue_date = "2025-01-01"
total_amount = "100.00"

invoice_manager = InvoiceManager()

Rechnung = invoice_manager.create_invoice(booking_id, "2025-1-1", 100.00)
if Rechnung is not None:
    print(
        f"Rechnung erstellt: {Rechnung.invoiceid}, BookingNr: {booking_id}, Ausstellungsdatum: {Rechnung.issue_date}, Gesamtbetrag: {Rechnung.total_amount}")
else:
    print("Es konnte keine Rechnung erstellt werden.")


3
Rechnung erstellt: (19, 1), BookingNr: 3, Ausstellungsdatum: 2025-1-1, Gesamtbetrag: 100.0


User-Story 6:
Als Gast möchte ich meine Buchung stornieren, damit ich nicht
belastet werde, wenn ich das Zimmer nicht mehr benötige.
Hint: Sorgt für die entsprechende Invoice.


In [2]:
# Fake Booking erstellen
from business_logic.booking_manager import BookingManager
from model.booking import Booking

valid = False
while not valid:
    input_user = input("Geben Sie bitte Ihren Booking ID ein: ")
    if len(input_user) > 0:
        valid = True
    else:
        print("Bitte geben Sie einen Booking ID ein.")
print(input_user)

booking_id = int(input_user)
booking_manager = BookingManager()

Stornierung = booking_manager.cancel_booking(booking_id)
print("Stornierung erfolgreich")


1


ValueError: Die Buchung mit der ID {booking_id} existiert nicht.